# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [50]:
# Creates a reference for the cities csv file and loads into a DataFrame

csv_path = "./cities_weather.csv"

df_cities_weather = pd.read_csv(csv_path)

df_cities_weather.head()



,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Wind Speed (mph),Cloudiness
0,Port Alfred,ZA,-33.59,26.89,61.93,87,16.22,78
1,Vaini,TO,-21.20,-175.20,75.20,88,19.46,75
2,New Norfolk,AU,-42.78,147.06,63.00,62,3.00,99
3,Nikolskoye,RU,59.70,30.79,37.00,94,6.71,75
4,Rikitea,PF,-23.12,-134.97,74.55,79,20.22,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
# Stores latitude and longitude from cities DataFrame in locations DataFrame
locations = df_cities_weather[["Latitude", "Longitude"]]

humidity = df_cities_weather["Humidity (%)"].astype(float)

locations

,Latitude,Longitude
0,-33.59,26.89
1,-21.20,-175.20
2,-42.78,147.06
3,59.70,30.79
4,-23.12,-134.97
...,...,...
583,18.90,-3.53
584,-22.92,-42.51
585,51.57,26.57
586,25.02,37.27


In [52]:
# Plot HeatMap

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
# Narrows down results of cities to fit the most excellent weather parameters

df_perf_humidity = df_cities_weather.loc[df_cities_weather["Humidity (%)"] < 75]

df_perf_temp = df_perf_humidity.loc[df_perf_humidity["Max Temp (F)"] < 95]

df_perf_temp = df_perf_temp.loc[df_perf_temp["Max Temp (F)"] > 75]

df_perf_clouds = df_perf_temp.loc[df_perf_temp["Cloudiness"] < 40]

df_perf_cities = df_perf_clouds.loc[df_perf_clouds["Wind Speed (mph)"] < 20]

df_perf_cities



,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Wind Speed (mph),Cloudiness
8,Busselton,AU,-33.65,115.33,78.01,32,2.89,0
9,Sokoto,NG,13.06,5.24,79.21,66,6.46,0
15,Nishihara,JP,26.18,127.76,84.20,54,12.75,20
55,Kavieng,PG,-2.57,150.80,85.06,71,6.73,22
60,Nizwá,OM,22.93,57.53,81.84,12,5.48,24
62,Diffa,NE,13.32,12.61,83.59,20,7.31,2
84,Noumea,NC,-22.28,166.46,80.60,47,13.87,20
85,Waingapu,ID,-9.66,120.26,87.49,53,10.67,6
105,Lorengau,PG,-2.02,147.27,85.69,64,9.28,10
119,Bairiki Village,KI,1.33,172.98,86.00,66,13.87,20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
# Creates data frame for storing ideal cities to search for hotels and adds a Hotel Name column for storing hotels

df_hotel = df_perf_cities.reset_index()

df_hotel["Hotel Name"] = ""
df_hotel["Lat"] = ""
df_hotel["Lng"] = ""

df_hotel



,index,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Wind Speed (mph),Cloudiness,Hotel Name,Lat,Lng
0,8,Busselton,AU,-33.65,115.33,78.01,32,2.89,0,,,
1,9,Sokoto,NG,13.06,5.24,79.21,66,6.46,0,,,
2,15,Nishihara,JP,26.18,127.76,84.20,54,12.75,20,,,
3,55,Kavieng,PG,-2.57,150.80,85.06,71,6.73,22,,,
4,60,Nizwá,OM,22.93,57.53,81.84,12,5.48,24,,,
5,62,Diffa,NE,13.32,12.61,83.59,20,7.31,2,,,
6,84,Noumea,NC,-22.28,166.46,80.60,47,13.87,20,,,
7,85,Waingapu,ID,-9.66,120.26,87.49,53,10.67,6,,,
8,105,Lorengau,PG,-2.02,147.27,85.69,64,9.28,10,,,
9,119,Bairiki Village,KI,1.33,172.98,86.00,66,13.87,20,,,


In [64]:
# # params dictionary to update each iteration
# params = {
#     "radius": 5000,
#     "types": "lodging",
#     "keyword": "hotel",
#     "key": gkey
# }

# lat = df_hotel["Latitude"][0]
# lng = df_hotel["Longitude"][0]

# # Changes location each iteration while leaving original params in place
# params["location"] = f"{lat},{lng}"

# # Use the search term: "hotel" and lat/lng from Hotels DataFrame
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # Makes request
# name_address = requests.get(base_url, params=params)

# # Converts to json
# name_address = name_address.json()

# # pprint(name_address)

# pprint(name_address["results"][0]["geometry"]["location"]["lat"])
# pprint(name_address["results"][0]["geometry"]["location"]["lng"])


-33.6469566
115.3454128


In [73]:
# Sets parameters to search for hotels with 5000 meters.

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

# Uses lat/lng from Hotels DataFrame
for index, row in df_hotel.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Changes location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and lat/lng from Hotels DataFrame
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Makes request
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

# Uses a try-except to skip any that are missing a data point.
    try:
        df_hotel.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        df_hotel.loc[index, "Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        df_hotel.loc[index, "Lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [80]:
# Converts hotel locations to floats and drops nans

df_hotel["Lat"] = pd.to_numeric(df_hotel["Lat"])
df_hotel["Lng"] = pd.to_numeric(df_hotel["Lng"])

print(df_hotel.dtypes)

df_hotel = df_hotel.dropna()

df_hotel


index                 int64
City                 object
Country              object
Latitude            float64
Longitude           float64
Max Temp (F)        float64
Humidity (%)          int64
Wind Speed (mph)    float64
Cloudiness            int64
Hotel Name           object
Lat                 float64
Lng                 float64
dtype: object


,index,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Wind Speed (mph),Cloudiness,Hotel Name,Lat,Lng
0,8,Busselton,AU,-33.65,115.33,78.01,32,2.89,0,Esplanade Hotel,-33.646957,115.345413
1,9,Sokoto,NG,13.06,5.24,79.21,66,6.46,0,Grand Ibro Hotel,13.060457,5.218983
2,15,Nishihara,JP,26.18,127.76,84.20,54,12.75,20,百名伽藍,26.132870,127.785433
3,55,Kavieng,PG,-2.57,150.80,85.06,71,6.73,22,Nusa Island Retreat,-2.582576,150.781292
4,60,Nizwá,OM,22.93,57.53,81.84,12,5.48,24,Antique Inn - Nizwa,22.932420,57.530487
5,62,Diffa,NE,13.32,12.61,83.59,20,7.31,2,Complexe Hôtelier Du 18 Décembre,13.312652,12.590391
6,84,Noumea,NC,-22.28,166.46,80.60,47,13.87,20,GONDWANA HOTEL,-22.276116,166.445465
7,85,Waingapu,ID,-9.66,120.26,87.49,53,10.67,6,Padadita Beach Hotel,-9.641084,120.270998
8,105,Lorengau,PG,-2.02,147.27,85.69,64,9.28,10,Lorengau Harbourside Hotel,-2.023093,147.270242
9,119,Bairiki Village,KI,1.33,172.98,86.00,66,13.87,20,Fema Lodge,1.340601,173.025565


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotel.iterrows()]
locations = df_hotel[["Lat", "Lng"]]

hotel_info

['\n<dl>\n<dt>Name</dt><dd>Esplanade Hotel</dd>\n<dt>City</dt><dd>Busselton</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Grand Ibro Hotel</dd>\n<dt>City</dt><dd>Sokoto</dd>\n<dt>Country</dt><dd>NG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>百名伽藍</dd>\n<dt>City</dt><dd>Nishihara</dd>\n<dt>Country</dt><dd>JP</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Nusa Island Retreat</dd>\n<dt>City</dt><dd>Kavieng</dd>\n<dt>Country</dt><dd>PG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Antique Inn - Nizwa</dd>\n<dt>City</dt><dd>Nizwá</dd>\n<dt>Country</dt><dd>OM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Complexe Hôtelier Du 18 Décembre</dd>\n<dt>City</dt><dd>Diffa</dd>\n<dt>Country</dt><dd>NE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>GONDWANA HOTEL</dd>\n<dt>City</dt><dd>Noumea</dd>\n<dt>Country</dt><dd>NC</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Padadita Beach Hotel</dd>\n<dt>City</dt><dd>Waingapu</dd>\n<dt>Country</dt><dd>ID</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Lor

In [87]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)

symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)

# Display figure

fig.add_layer(markers)
fig.add_layer(symbol_layer)
fig

Figure(layout=FigureLayout(height='420px'))